In [1]:
%load_ext autoreload
%autoreload 2

# Maserati Data

In [2]:
from rdkit import Chem
import pandas as pd
import os, pickle, gzip, zipfile, shutil
import subprocess as sp

### Clean working directory

In [3]:
names = []
for root, dirs, files in os.walk('work'):
    for f in files:
        if f.endswith('_charged.mol2'):
            names.append(f'{root}/{f}')
        if f.endswith('_apo.pdb'):
            names.append(f'{root}/{f}')
        if f.endswith('_apo.mol2'):
            names.append(f'{root}/{f}')
for fname in names:
    os.unlink(fname)

### For GNN-DTI

In [4]:
todo = []
for root, dirs, files in os.walk('work'):
    ligand_name = None
    pdb_code = None
    for f in files:
        if f.endswith('.sdf') and len(f) == 7:
            ligand_name = f[:3]
            pdb_code = os.path.basename(root)
            break
    if not pdb_code or not ligand_name:
        continue
    todo.append((pdb_code, ligand_name))
len(todo)

2503

### For GNN-DTI Essentials (Hasegawa selection)

In [6]:
os.makedirs('2018', exist_ok=True)
os.makedirs('select', exist_ok=True)

df = pd.read_pickle('index_2019.pkl.gz')

counts = {'select': 0, '2018': 0}

for pdb_code, ligand_name in todo:
    r = df[df['pdb'] == pdb_code]
    year = r['year'].item()
    ligand_name = r['lig'].item()
    select = r['select'].item()

    if year == 2018:
        counts['2018'] += 1
        dest = f'2018/{pdb_code}'
    else:
        counts['select'] += 1
        dest = f'select/{pdb_code}'

    pocket_fname = f'work/{pdb_code}/{pdb_code}_pocket.pdb'
    ligand_fname = f'work/{pdb_code}/{ligand_name}.sdf'
    value_fname = f'work/{pdb_code}/value'
    
    os.makedirs(dest, exist_ok=True)
    shutil.copy(pocket_fname, dest)
    shutil.copy(ligand_fname, dest)
    shutil.copy(value_fname, dest)

sp.call(f'tar jcvf cbidata_gnn_dti.tar.bz2 2018/ select/', shell=True)

shutil.rmtree('2018')
shutil.rmtree('select')
counts

{'select': 2082, '2018': 421}